# Trust Region CG-Newton Method

Before we begin our disscusion of conjugate gradient quasi-newton trust regions methods we begin with a newton direction in CG routine that was introduced by Steihaug.
The use of a CG scheme is in our trust-region subproblem, namaley, when we try to minimize our model function over a region defined by our TR radius $\Delta_k$.
Specifically, we use a newton CG to solve:  

$$ \min_{p \in \mathbb{R}} m_k(p) = f_k + g_k^T p + \frac12 p^T B_k p ~ : ||p|| \leq \Delta_K, $$

where $g_k = \nabla f(x_k)$ and $B_k = \nabla^2 f(x_k)$ (the Newton direction). This problem is solved in `cg_steihaug` below.

In [1]:
using LinearAlgebra, ForwardDiff

function cg_steihaug(f, x, Δ, ϵ) #Alg 7.2
    ∇f = dx -> ForwardDiff.gradient(f, dx)
    ∇²f = dx -> ForwardDiff.hessian(f, dx)
    print("hi")
    d = -∇f(x)
    B = ∇²f(x)
    d_norm = norm(d)
    
    # Arrived at an optimal iterate x minimizing f?
    if d_norm < ϵ
        return p
    end 
    
    z = 0 # cg direction
    r = -d
    # solve tr subproblem by conjugate gradient
    while true
        if (α = d'*B*d) <= 0   # g is a direction of neg. curvature
            return  d*Δ/d_norm
        end
        
        temp = r'*r
        z = z + temp/α * d
        if (z_norm = norm(z)) >= Δ
             return  z*Δ/z_norm
        end

        r = r + α*B*d
        if (r_norm = norm(r)) <= ϵ
            return z
        end
        d = -r + r_norm^2/temp * d
    end
end 

cg_steihaug (generic function with 1 method)

# Quasi-Newton Methods

These methods are classified by their unique ability to require only the gradient of the objective function at each iteration.
By measuring the change in gradients between iterations, it can construct a model of the objective function, $f$, that produces superlinear convergence. We begin with the derivation of a Quasi-Newton approach by constructing a quadratic model of $f$ at an iterate $x_k$:  

$$ m_k(p) = f_k + \nabla f_K^T p + \frac12 p^T B_K p, $$

where $B_k$ is a square SPD matrix that will be updated at every iteration.

pg. 146, safegaurd sr1, veryify with srmin  
pg. 8 Fundamentals of Matrix Computations.... Block proofs needed to put algs in block form.